## 安装 LLaMA Factory 依赖

In [ ]:
%cd /content/
%rm -rf LLaMA-Factory
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
%cd LLaMA-Factory
%ls
!pip install pydantic==2.10.6
!pip install -e .[torch,bitsandbytes]
# !pip install --no-deps -e .

/content
Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 345, done.
remote: Counting objects: 100% (345/345), done.
remote: Compressing objects: 100% (281/281), done.
remote: Total 345 (delta 82), reused 172 (delta 48), pack-reused 0 (from 0)
Receiving objects: 100% (345/345), 9.54 MiB | 14.56 MiB/s, done.
Resolving deltas: 100% (82/82), done.
/content/LLaMA-Factory
assets/       docker/      LICENSE      pyproject.toml  requirements.txt  src/
CITATION.cff  evaluation/  Makefile     README.md       scripts/          tests/
data/         examples/    MANIFEST.in  README_zh.md    setup.py
Obtaining file:///content/LLaMA-Factory
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━

## 使用 LLaMA Board Web UI 微调模型

In [ ]:
%cd /content/LLaMA-Factory/
!GRADIO_SHARE=1 llamafactory-cli webui

/content/LLaMA-Factory
2025-03-30 10:49:38.492313: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-30 10:49:38.512043: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743331778.534768   20428 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743331778.541704   20428 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-30 10:49:38.565397: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to 

## 使用命令行微调模型

微调过程大约需要 30 分钟。

In [ ]:
import json

args = dict(
  stage="sft",                                               # 进行指令监督微调
  do_train=True,
  model_name_or_path="Qwen/Qwen2.5-7B-Instruct",
  dataset="yue_1,yue_2",
  template="alpaca",                                         # 使用 llama3 提示词模板
  finetuning_type="lora",                                    # 使用 LoRA 适配器来节省显存
  lora_target="all",                                         # 添加 LoRA 适配器至全部线性层
  output_dir="Qwen2.5_lora",                                  # 保存 LoRA 适配器的路径
  per_device_train_batch_size=2,                             # 批处理大小
  gradient_accumulation_steps=4,                             # 梯度累积步数
  lr_scheduler_type="cosine",                                # 使用余弦学习率退火算法
  logging_steps=5,                                           # 每 5 步输出一个记录
  warmup_ratio=0.1,                                          # 使用预热学习率
  save_steps=1000,                                           # 每 1000 步保存一个检查点
  learning_rate=5e-5,                                        # 学习率大小
  num_train_epochs=1.0,                                      # 训练轮数
  max_samples=1000,                                           # 使用每个数据集中的 10000 条样本
  max_grad_norm=1.0,                                         # 将梯度范数裁剪至 1.0
  loraplus_lr_ratio=16.0,                                    # 使用 LoRA+ 算法并设置 lambda=16.0
  fp16=True,                                                 # 使用 float16 混合精度训练
  report_to="none",                                          # 关闭 wandb 记录器
)

json.dump(args, open("train_llama3.json", "w", encoding="utf-8"), indent=2)

%cd /content/LLaMA-Factory/

!llamafactory-cli train train_llama3.json

/content/LLaMA-Factory
2025-03-30 11:42:18.604754: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-30 11:42:18.624121: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743334938.646635   33719 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743334938.653442   33719 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-30 11:42:18.677164: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to 

## 模型推理

In [ ]:
import sys
sys.path.append('/content/LLaMA-Factory/src')
from llamafactory.chat import ChatModel
from llamafactory.extras.misc import torch_gc

%cd /content/LLaMA-Factory/

args = dict(
  model_name_or_path="Qwen/Qwen2.5-7B-Instruct", # 使用 4 比特量化版 Llama-3-8b-Instruct 模型
  adapter_name_or_path="llama3_lora",                        # 加载之前保存的 LoRA 适配器
  template="llama3",                                         # 和训练保持一致
  finetuning_type="lora",                                    # 和训练保持一致
)
chat_model = ChatModel(args)

messages = []
print("使用 `clear` 清除对话历史，使用 `exit` 退出程序。")
while True:
  query = input("\nUser: ")
  if query.strip() == "exit":
    break
  if query.strip() == "clear":
    messages = []
    torch_gc()
    print("对话历史已清除")
    continue

  messages.append({"role": "user", "content": query})
  print("Assistant: ", end="", flush=True)

  response = ""
  for new_text in chat_model.stream_chat(messages):
    print(new_text, end="", flush=True)
    response += new_text
  print()
  messages.append({"role": "assistant", "content": response})

torch_gc()

## 合并 LoRA 权重并上传模型

注意：Colab 免费版仅提供了 12GB 系统内存，而合并 8B 模型的 LoRA 权重需要至少 18GB 系统内存，因此你 **无法** 在免费版运行此功能。

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write

In [ ]:
import json

args = dict(
  model_name_or_path="Qwen/Qwen2.5-7B-Instruct", # 使用非量化的官方 Llama-3-8B-Instruct 模型
  adapter_name_or_path="Qwen2.5_lora",                       # 加载之前保存的 LoRA 适配器
  template="alpaca",                                        # 和训练保持一致
  finetuning_type="lora",                                   # 和训练保持一致
  export_dir="Qwen_lora_merged",                          # 合并后模型的保存目录
  export_size=2,                                            # 合并后模型每个权重文件的大小（单位：GB）
  export_device="cpu",                                      # 合并模型使用的设备：`cpu` 或 `auto`
  export_hub_model_id="Nin8520/Qwen2.5-7B",               # 用于上传模型的 HuggingFace 模型 ID
)

json.dump(args, open("merge_llama3.json", "w", encoding="utf-8"), indent=2)

%cd /content/LLaMA-Factory/

!llamafactory-cli export merge_llama3.json

Streaming output truncated to the last 5000 lines.




model-00002-of-00009.safetensors:  24% 451M/1.86G [00:25<00:51, 27.7MB/s]




model-00008-of-00009.safetensors:  33% 351M/1.07G [00:25<00:19, 36.1MB/s]



model-00001-of-00009.safetensors:  20% 368M/1.89G [00:25<01:02, 24.5MB/s]



model-00001-of-00009.safetensors:  20% 382M/1.89G [00:25<00:41, 36.1MB/s]

model-00004-of-00009.safetensors:  12% 225M/1.86G [00:25<05:05, 5.36MB/s]




model-00002-of-00009.safetensors:  25% 471M/1.86G [00:25<00:49, 28.1MB/s]
model-00006-of-00009.safetensors:  12% 220M/1.86G [00:25<03:38, 7.53MB/s]

model-00004-of-00009.safetensors:  12% 233M/1.86G [00:25<02:07, 12.8MB/s]




model-00002-of-00009.safetensors:  26% 479M/1.86G [00:25<00:40, 34.3MB/s]

model-00008-of-00009.safetensors:  34% 358M/1.07G [00:26<00:25, 27.7MB/s]
model-00006-of-00009.safetensors:  12% 222M/1.86G [00:26<03:31, 7.78MB/s]

model-00008-of-00009.safetensors:  34% 366M/1.07G [00:26<00:21, 32.8MB/s]
model-00006-of-00009.safetensors:  